# CPTox21: Conformal prediction and exchangeability in in vitro toxicological datasets

This notebooks demonstrates the main workflow to obtain the results for the manuscript ...

### Import libraries and modules

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.svm import SVC
from nonconformist.nc import NcFactory, MarginErrFunc

In [3]:
from cptox21 import (
    define_path, load_signatures_files, StratifiedRatioSampler,CrossValidationSampler,
    KnownIndicesSampler, InductiveConformalPredictor,
    AggregatedConformalPredictor, CPTox21AggregatedConformalPredictor, 
    CPTox21CrossValidator, CPTox21TrainUpdateCrossValidator
)

### Define paths and parameters

In [4]:
data_signatures_path = "../data/data_signatures/"
# data_models_path = args["data_models_path"]
data_statistics_path = "../data/data_statistics/"

In [5]:
endpoint = "NR_ER"

In [6]:
# CP
n_folds_acp = 2

#### Build ICP classifier

In [7]:
clf = SVC(kernel='rbf', C=50, gamma=0.002, probability=True)
error_function = MarginErrFunc()
normaliser_model = None
nc = NcFactory.create_nc(
    clf, err_func=error_function
)
icp = InductiveConformalPredictor(
        nc_function=nc, condition=(lambda instance: instance[1])
    )  # Mondrian as (default) condition)

#### Build ACP for main framework and for train_update

In [8]:
acp = CPTox21AggregatedConformalPredictor(
        predictor=icp, sampler=StratifiedRatioSampler(n_folds=n_folds_acp),
        aggregation_func=np.median
    )

#### Load data

In [9]:
datasets = ["train", "test", "score"]
train_path = define_path(endpoint=endpoint, data=datasets[0], signatures_path=data_signatures_path)
test_path = define_path(endpoint=endpoint, data=datasets[1], signatures_path=data_signatures_path)
score_path = define_path(endpoint=endpoint, data=datasets[2], signatures_path=data_signatures_path)

X_train, y_train, X_test, y_test, X_score, y_score = load_signatures_files(train_path, test_path, score_path)
# X_train = X_train[:500]
# y_train = y_train[:500]

#### Define crossvalidator, which does all acp experiments except for train_update

In [10]:
cross_validator = CPTox21CrossValidator(
            acp, cv_splitter=CrossValidationSampler(),
            score_splitter=StratifiedRatioSampler(test_ratio=0.5)
        )

#### Crossvalidate

In [ ]:
cross_validator.cross_validate(
            steps=10,
            endpoint=endpoint,
            X_train=X_train,
            y_train=y_train,
            X_update=X_test,
            y_update=y_test,
            X_score=X_score,
            y_score=y_score,
        )

len score 245 245


<__array_function__ internals>:5: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.


[[0.54732972 0.23387861]
 [0.93837149 0.03139152]
 [0.53043337 0.2401428 ]
 ...
 [0.7435561  0.13300185]
 [0.90327285 0.05879658]
 [0.76718295 0.12389004]] <class 'numpy.ndarray'>
(1439, 3)
(1439, 3)
[[2.34582202e-01 4.00530071e-01]
 [5.19391394e-01 2.35822968e-01]
 [5.58745363e-02 5.70369173e-01]
 [7.25308282e-01 1.39740297e-01]
 [7.82417536e-01 1.34975341e-01]
 [3.47826715e-02 6.28640605e-01]
 [1.20760737e-01 4.82577420e-01]
 [4.62950050e-01 2.67130264e-01]
 [1.43229770e-01 4.82448354e-01]
 [5.54521310e-01 2.38094706e-01]
 [9.98261127e-02 5.11105367e-01]
 [5.59303499e-01 2.14888459e-01]
 [2.84391974e-01 3.89500817e-01]
 [8.50922454e-01 8.48766586e-02]
 [7.25570779e-02 5.47717218e-01]
 [1.24809846e-02 7.50865524e-01]
 [7.50099115e-01 1.29127178e-01]
 [3.65202782e-01 3.45745059e-01]
 [3.06069411e-01 3.57675409e-01]
 [4.22938982e-01 2.88612096e-01]
 [7.52280364e-01 1.20835158e-01]
 [5.22601624e-01 2.57798505e-01]
 [6.31919226e-01 1.90754839e-01]
 [7.45742926e-01 1.29163389e-01]
 [7.1179

[[0.36394539 0.42375499]
 [0.57564988 0.28976608]
 [0.11797538 0.69696361]
 [0.78762512 0.1814846 ]
 [0.82470862 0.11577767]
 [0.08269045 0.75512377]
 [0.20624003 0.54965469]
 [0.55704688 0.34726824]
 [0.25207528 0.58466001]
 [0.61543223 0.23379543]
 [0.17304263 0.61538425]
 [0.61892223 0.25216353]
 [0.35332486 0.49482374]
 [0.87264813 0.08867031]
 [0.15351413 0.68160439]
 [0.01580332 0.93917462]
 [0.80602361 0.15703331]
 [0.4472427  0.29141656]
 [0.40973344 0.39224846]
 [0.51097591 0.37498225]
 [0.77469493 0.14864013]
 [0.60491346 0.2184822 ]
 [0.67841195 0.19558978]
 [0.78467415 0.18056383]
 [0.76531192 0.15657476]
 [0.77637063 0.17513023]
 [0.45608166 0.2880087 ]
 [0.69203178 0.21160032]
 [0.5006137  0.34988221]
 [0.70754492 0.19886233]
 [0.59834265 0.2408467 ]
 [0.96704334 0.02571093]
 [0.50513684 0.37496378]
 [0.61555769 0.2799584 ]
 [0.32939801 0.43714086]
 [0.29325272 0.49049929]
 [0.43060846 0.34909059]
 [0.8261465  0.13680684]
 [0.92457747 0.09236083]
 [0.19505558 0.63239264]


len score 245 245


...

<__array_function__ internals>:5: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.


[[0.36180674 0.31229809]
 [0.41953649 0.37605319]
 [0.34561257 0.31756392]
 ...
 [0.50040255 0.22424158]
 [0.11747369 0.46433835]
 [0.68357997 0.15308182]] <class 'numpy.ndarray'>
(1438, 3)
(1438, 3)
[[1.74524276e-01 4.15367171e-01]
 [5.59423003e-01 2.06594913e-01]
 [3.13575850e-01 3.37808141e-01]
 [2.70935028e-01 3.97125033e-01]
 [7.18677389e-01 1.38899061e-01]
 [4.02343963e-02 5.84805950e-01]
 [2.07298935e-01 3.95264181e-01]
 [6.21194562e-01 1.88891288e-01]
 [5.46851107e-01 2.12690687e-01]
 [6.23560025e-01 1.87465799e-01]
 [1.21069468e-01 4.50366403e-01]
 [5.82498443e-01 1.94109757e-01]
 [1.25008424e-01 4.56716787e-01]
 [4.35894674e-02 5.74144325e-01]
 [9.75273847e-02 5.11548458e-01]
 [2.39281914e-02 6.56595611e-01]
 [9.06698032e-01 4.83655396e-02]
 [4.69520295e-01 2.43513093e-01]
 [1.84852658e-01 4.10774032e-01]
 [5.83970334e-01 2.05135204e-01]
 [7.96347886e-01 1.13296505e-01]
 [4.21999850e-01 2.94254341e-01]
 [3.13899822e-01 3.34031841e-01]
 [2.16170314e-01 3.97479686e-01]
 [7.1324

[[3.32402396e-01 4.52962680e-01]
 [6.15182353e-01 2.35565467e-01]
 [4.41850126e-01 3.83284742e-01]
 [3.26690779e-01 5.14858604e-01]
 [7.63867476e-01 1.48688724e-01]
 [7.25782269e-02 7.74647175e-01]
 [3.69811957e-01 4.71464078e-01]
 [6.81740379e-01 2.02340893e-01]
 [6.19196096e-01 2.82949324e-01]
 [7.08994016e-01 1.77713903e-01]
 [2.51298416e-01 5.13483678e-01]
 [6.34603520e-01 2.46434571e-01]
 [2.63083971e-01 5.00793595e-01]
 [8.55324676e-02 7.57064672e-01]
 [1.98274343e-01 6.44022589e-01]
 [2.89589427e-02 9.18047836e-01]
 [9.34155706e-01 1.99195789e-02]
 [5.57498339e-01 2.97161545e-01]
 [3.47831898e-01 4.74297308e-01]
 [6.48477638e-01 2.36152360e-01]
 [8.43628048e-01 1.00135025e-01]
 [5.44086814e-01 3.54315008e-01]
 [4.36683430e-01 3.79769959e-01]
 [3.76875646e-01 3.87174543e-01]
 [7.58354008e-01 1.63548134e-01]
 [8.19756593e-01 1.06913876e-01]
 [6.68966926e-01 2.18994366e-01]
 [6.69775398e-01 2.10094001e-01]
 [6.42065544e-01 2.42700764e-01]
 [7.22776826e-01 1.77465133e-01]
 [6.842575

len score 245 245


<__array_function__ internals>:5: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.


#### Get splits from crossvalidator, so that exactly the same splits can be used for train_update

In [ ]:
train_index, test_index = cross_validator.train_indices, cross_validator.test_indices

known_indices_sampler = KnownIndicesSampler(known_train=train_index, known_test=test_index)



#### Define ACP for train_update

In [ ]:
train_update_acp = AggregatedConformalPredictor(
        predictor=icp, sampler=StratifiedRatioSampler(n_folds=n_folds_acp),
        aggregation_func=np.median
    )

#### Define train_update crossvalidator using same splits as before and crossvalidate

In [ ]:
train_update_cross_validator = CPTox21TrainUpdateCrossValidator(train_update_acp, cv_splitter=known_indices_sampler)

train_update_cross_validator.cross_validate(steps=10,
                                           endpoint=endpoint,
                                           X_train=X_train,
                                           y_train=y_train,
                                           X_update=X_test,
                                           y_update=y_test,
                                           X_score=X_score,
                                           y_score=y_score,
                                           class_wise_evaluation=False)

### Plots

In [ ]:
plots = {}

In [ ]:
plots["cv"] = cross_validator.calibration_plot( averaged_evaluation_df=cross_validator.averaged_evaluation_df_cv,
            endpoint=endpoint,
        )
plots["pred_score"] = cross_validator.calibration_plot(
            averaged_evaluation_df=cross_validator.averaged_evaluation_df_pred_score,
            endpoint=endpoint,
        )
plots["pred_test"] = cross_validator.calibration_plot(
            averaged_evaluation_df=cross_validator.averaged_evaluation_df_pred_test,
            endpoint=endpoint,
        )
plots["cal_update"] = cross_validator.calibration_plot(
            averaged_evaluation_df=cross_validator.averaged_evaluation_df_pred_cal_update,
            endpoint=endpoint,
        )
plots["cal_update2"] = cross_validator.calibration_plot(
            averaged_evaluation_df=cross_validator.averaged_evaluation_df_pred_cal_update2,
            endpoint=endpoint,
        )
plots["train_update_cv"] = train_update_crossvalidator.calibration_plot(
            averaged_evaluation_df=train_update_crossvalidator.averaged_evaluation_df_cv,
            endpoint=endpoint,
        )
plots["train_update_pred_score"] = train_update_crossvalidator.calibration_plot(
            averaged_evaluation_df=train_update_crossvalidator.averaged_evaluation_df_pred_score,
            endpoint=endpoint,
        )